### Retina hub - a user driven database of papers in the retina field

#### This notebook contains code to populate the retina paper database.

- An online form is presented to users. 
- They fill out information about the paper they want to add (URL, DOI and keywords)
- code pulls information from the form (results are stored in a datasheet) and stores it in pandas
- using CrossREF api (and habanero), we use DOIs to retrive metadata (authors, publication year, journal etc)
- using Zotero api (and pyzotero), we put the metadata retrieved into a Zotero public library, together with keywords added by users
- using anothe python library we use each zotero entry to create a post on a wordpress website.



In [6]:
#import all necessary libraries
import pandas as pd
from pyzotero import zotero

In [7]:
#import data from datasheet
with open("zotero_sheet.txt") as fid:
    sheetId = fid.readline()
    

#pull data from the sheet into a pandas dataframe
allPosts = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   sheetId +
                   '/export?format=csv',
                   # Set first column as rownames in data frame
                   index_col=0,
                  )


In [31]:
#play around with zotero api and see what is in the zotero library
with open("zotero_key.txt") as fid:
    apiKey = fid.readline()
    
libID = "4584648"
libType = "group"

zot = zotero.Zotero(libID, libType, apiKey)
#zot.top gets the first 100 items in the library
items = zot.top()
#this is how to get all of them
results = zot.everything(zot.top()).

# we've retrieved the latest five top-level items in our library
# we can print each item's item type and ID
for item in items:
    print('Item: %s | Key: %s' % (item['data']['itemType'], item['data']['key']))

Item: journalArticle | Key: 6PR8W7IH
Item: journalArticle | Key: 66FP7PBR
Item: journalArticle | Key: 86IJ7FFR
Item: journalArticle | Key: 9ZDTC2F2
Item: journalArticle | Key: DSU9R3VV
Item: journalArticle | Key: 2QCMBMFE
Item: journalArticle | Key: XZM6JBVI
Item: journalArticle | Key: UXSDA6HP
Item: report | Key: ZC6VUGHD
Item: journalArticle | Key: 76B3GHUK
Item: report | Key: 3WWIJHWR
Item: journalArticle | Key: QI3AHQS7
Item: journalArticle | Key: KGEVRVAF
Item: journalArticle | Key: 4ZZ87KXW
Item: report | Key: BNJB4AA8
Item: journalArticle | Key: FD8ZWASG
Item: report | Key: ZKDS6WN5
Item: journalArticle | Key: M98CM6UE
Item: journalArticle | Key: 3VJWM4D3


In [92]:
#example on how to get metadata based on DOI

from crossref.restful import Works
works = Works()
test = works.title("The temporal structure of the inner retina at a single glance")


AttributeError: 'Works' object has no attribute 'title'

In [93]:
vars(works)

{'do_http_request': <bound method HTTPRequest.do_http_request of <crossref.restful.HTTPRequest object at 0x7fd5992bc5b0>>,
 'etiquette': <crossref.restful.Etiquette at 0x7fd5992bc220>,
 'custom_header': {'user-agent': 'undefined/undefined (undefined; mailto:anonymous) BasedOn: CrossrefAPI/1.5.0'},
 'crossref_plus_token': None,
 'request_url': 'https://api.crossref.org/works',
 'request_params': {},
 'context': ''}

In [59]:
template = zot.item_template('journalArticle')
template["DOI"]='10.1016/j.cub.2020.05.055'

In [60]:
#create an item for the zotero library
resp = zot.create_items([template])

In [91]:
from habanero import Crossref
cr = Crossref()
x = cr.works(title = "The temporal structure of the inner retina at a single glance", author="zhao")
x['message']
x['message']['total-results']
x['message']['items']

[{'indexed': {'date-parts': [[2021, 5, 9]],
   'date-time': '2021-05-09T15:42:27Z',
   'timestamp': 1620574947097},
  'reference-count': 26,
  'publisher': 'LLC CPC Business Perspectives',
  'issue': '3',
  'license': [{'start': {'date-parts': [[2018, 10, 4]],
     'date-time': '2018-10-04T00:00:00Z',
     'timestamp': 1538611200000},
    'content-version': 'unspecified',
    'delay-in-days': 0,
    'URL': 'http://creativecommons.org/licenses/by/3.0/deed.en_US'}],
  'content-domain': {'domain': [], 'crossmark-restriction': False},
  'short-container-title': ['Banks and Bank Systems'],
  'published-print': {'date-parts': [[2018, 10, 4]]},
  'abstract': '<jats:p>The study intends to explore the acceptance model of Syariah banking in Bali, Indonesia. It considers that local values, especially menyama-braya, are one of the local wisdoms, and they might encourage people to accept Syariah banks, since Syariah banking emphasizes harmonious relations. Environment marketing theory is useful to 